### file 읽어오기

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import TextLoader
path = r'data/medicine.txt'


In [5]:
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

set_llm_cache(SQLiteCache(".cache_prompt.sqlite"))

COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db"
EMBEDDING_MODEL_NAME = 'text-embedding-3-small'
from langchain_openai import OpenAIEmbeddings

# Text Loading
loader = TextLoader(path, encoding='utf-8')

# 문서 load and split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(splitter)

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# vector_store = Chroma(
#     embedding_function=embedding_model,
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY
# )

retriever = vector_store.as_retriever(
    search_type='mmr',
    # search_type="similarity_score_threshold",
    search_kwargs = {
        'k':5,
        'fetch_k':10
    }
)



In [6]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=2,
        return_messages=True,
        message_key="history"
    )
    message_list = memory.load_memory_variables({})['history']
    store[session_id] = InMemoryChatMessageHistory(messages=message_list)
    return store[session_id]

In [7]:
from geopy.distance import geodesic
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import json


def read_file() :
    # CSV 파일 경로 설정
    pharmacy_data_path = r"data/pharmacy.csv"

    # CSV 파일 읽기
    try:
        return pd.read_csv(pharmacy_data_path, encoding='utf-8')
    except Exception as e:
        return None

pharmacy_data = read_file()

# 사용자 IP를 통해 위치 가져오기
def get_user_location_by_ip():
    try:
        load_dotenv()
        google_api_key = os.environ.get('GOOGLE_API_KEY')
        url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={google_api_key}'

        response = json.loads(requests.post(url).text)
        
        location = response['location']
        lat = round(location['lat'], 2)
        lng = round(location['lng'], 2)
        lat_lng = (lat, lng)
        
        return lat_lng
    except Exception as e:
        print("IP 기반 위치를 가져오는 데 실패했습니다.", str(e))
        return None

# 시간 변환 함수
def convert_time_format(time):
    """900 → 09:00 형식으로 변환"""
    try:
        time = str(int(time))  # 정수로 변환 후 문자열로 처리
        if len(time) == 3:  # 3자리인 경우 (예: 900)
            time = f"0{time[0]}:{time[1:]}"
        elif len(time) == 4:  # 4자리인 경우 (예: 1830)
            time = f"{time[:2]}:{time[2:]}"
        return time
    except ValueError:
        return "정보 없음"

# 운영시간 변환 함수
def format_operating_hours(row, day):
    """운영시간을 시작~종료 형식으로 변환"""
    start_time = row.get(f'진료시간({day})S', '정보 없음')  # 시작 시간
    close_time = row.get(f'진료시간({day})C', '정보 없음')  # 종료 시간
    
    if start_time != '정보 없음' and close_time != '정보 없음':
        return f"{convert_time_format(start_time)}~{convert_time_format(close_time)}"
    else:
        return "정보 없음"

# 근처 약국 추천 함수
def recommend_nearby_pharmacies(user_location, max_distance_km=1, num_pharmacies=2):
    if pharmacy_data is None:
        print("약국 데이터가 없습니다. CSV 파일을 확인하세요.")
        return []

    recommended_pharmacies = []

    for _, row in pharmacy_data.iterrows():
        try:
            # 위도와 경도 열 이름 확인
            pharmacy_location = (float(row['병원위도']), float(row['병원경도']))
            distance = geodesic(user_location, pharmacy_location).km

            if distance <= max_distance_km:
                recommended_pharmacies.append({
                    "name": row['약국명'],
                    "address": row['주소'],
                    "phone": row.get('대표전화1', '정보 없음'),
                    "distance": round(distance, 2),
                    "hours": {
                        "weekday": format_operating_hours(row, '월요일'),
                        "saturday": format_operating_hours(row, '토요일'),
                        "sunday": format_operating_hours(row, '일요일')
                    }
                })
        except Exception as e:
            print("약국 데이터 처리 중 오류:", str(e))
            continue

    # 거리순으로 정렬 후 상위 n개 선택
    recommended_pharmacies = sorted(recommended_pharmacies, key=lambda x: x['distance'])[:num_pharmacies]

    return recommended_pharmacies


def recommend_pharmacy() :
    # 사용자 위치 가져오기 및 약국 추천 실행
    if pharmacy_data is not None:
        user_location = get_user_location_by_ip()
        text = ''
        if user_location:
            nearby_pharmacies = recommend_nearby_pharmacies(user_location)

            # 결과 출력
            if nearby_pharmacies:
                for pharmacy in nearby_pharmacies:
                    text += f"{pharmacy['name']} {pharmacy['address']} {pharmacy['phone']} {pharmacy['distance']} km 평일: {pharmacy['hours']['weekday']} 토요일: {pharmacy['hours']['saturday']} 일요일: {pharmacy['hours']['sunday']}\n"
            else:
                text = "근처에 추천할 약국이 없습니다."
        else:
            text = "사용자 위치를 가져오는 데 실패했습니다."

        summary_prompt = PromptTemplate(
            template="근처 약국에 대한 내용이야. 다음 내용을 요약해줘.\n[요약할 내용]\n{content}"
        )
        summary_model = ChatOpenAI(model="gpt-4o-mini")
        summary_chain = summary_prompt | summary_model | StrOutputParser()
        summary_result = summary_chain.invoke({"content":text})
        return summary_result
    else:
        print("약국 데이터를 로드하지 못했습니다. 파일 경로와 내용을 확인하세요.")

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter
from langchain_openai import ChatOpenAI

# Prompt Template 생성
messages = [
            ("ai", """
            너는 유능한 약사야. context 안에 있는 약 중에 사용자가 입력한 증상에 맞는 약을 3개만 선택해서 어떤 효과가 있는지 알려줘.
            복용이 간단하고 부작용이 적은 약을 우선적으로 추천해서 알려줘.
            약을 복용하기 전에 반드시 알아야 할 사항은 꼭 알려줘.
            증상이 악화될 경우 대처방법에 대해서 알려줘.
            약에 따른 복용법에 대해서도 간단히 언급해줘.
            주의사항 요약해서 알려줘. 
            위의 내용을 표로 정리해서 알려줘.
            사용자가 말한 증상을 효능 부분에서 찾아서 그에 맞는 약을 알려줘.

            그리고 약국에 대한 정보를 물어보면 'recommend_pharmacy'라고만 트리거 줘
            context가 없으면 모른다고해
            
            [context]
            {context}")"""),
            MessagesPlaceholder("history"), 
            ("human", "{question}")
    ]

prompt_template = ChatPromptTemplate(messages)

model = ChatOpenAI(model='gpt-4o-mini')

parser = StrOutputParser()

def get_context(input_data:dict):
    return retriever.invoke(input_data['question'])

runnable = {'context':RunnableLambda(get_context), 'question':itemgetter("question"), 'history':itemgetter('history')} | prompt_template | model | parser

chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

config = {"configurable": {"session_id":"id-1"}}


In [10]:
input_message = input("Question:")
while True:
    if input_message == "q!":
        break
    data = chain.invoke({"question":input_message}, config)
    if data == 'recommend_pharmacy' :
        data = recommend_pharmacy()
    print(input_message)
    print('-'*200)
    print(data)
    input_message = input("Question:")


두통에 좋은 약 추천해줘
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
아래는 두통에 효과적인 약 3가지와 그에 대한 정보입니다.

| 약품명                | 효능                                              | 복용법                                           | 주의사항 및 대처 방법                                                                                              |
|---------------------|-------------------------------------------------|------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| 속콜펜정           | 두통, 치통, 관절통 등 다양한 통증 완화 및 해열 | 만 15세 이상: 1회 1정, 1일 3회 (공복 피하기)  | 과민증 환자, 만 3개월 미만 영아, 알레르기 체질, 간장/신장/심장 질환자, 임부/수유부는 사용 전에 상담 필요. 증상 5~6회 복용 후 개선 없으면 중지하고 상담. |
| 이부플러스생정     | 두통, 생리통 등 통증 완화 및 해열                | 만 15세 이상: 1회 2정, 1일 3회 (공복 피하기)  | 과민증 환자, 만 3개월 

/var/folders/8y/jsfncy6n39v2jylvdpm61nmh0000gn/T/ipykernel_18627/1469145467.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


치통에 좋은 약 추천해줘
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
아래는 치통에 효과적인 약 3가지와 그에 대한 정보입니다.

| 약품명                | 효능                                              | 복용법                                           | 주의사항 및 대처 방법                                                                                              |
|---------------------|-------------------------------------------------|------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| 속콜펜정           | 두통, 치통, 관절통 등 다양한 통증 완화 및 해열 | 만 15세 이상: 1회 1정, 1일 3회 (공복 피하기)  | 과민증 환자, 만 3개월 미만 영아, 알레르기 체질, 간장/신장/심장 질환자, 임부/수유부는 사용 전에 상담 필요. 증상 5~6회 복용 후 개선 없으면 중지하고 상담. |
| 이부플러스생정     | 치통, 두통, 생리통 등 통증 완화 및 해열        | 만 15세 이상: 1회 2정, 1일 3회 (공복 피하기)  | 과민증 환자, 만 3개월 미만 영